In [1]:
import time
import random
import os
import pandas
import logging

from functools import wraps


def timer(func):
    @wraps(func)
    def wrapper(*args):
        logger = logging.getLogger('main')
        st = time.perf_counter()
        result = func(*args)
        end = time.perf_counter()
        internal = int((end - st)*1000)
        logger.debug(f'{func.__name__} cost {internal} ms')
        return result

    return wrapper


def print_seperator(func):
    def wrapper(*argc, **kwargs):
        print('-' * 90)
        ret = func(*argc, **kwargs)
        print('-' * 90)
        print()
        return ret
    return wrapper


def delay(delta: int):
    rand = random.random()
    rand *= delta
    logging.getLogger().warning(f'sleep {rand} s')
    time.sleep(rand)


def collect_from_dict(dict, lst) -> list:
    return [dict.get(item) for item in lst]


def check_dict(dict, keys) -> bool:
    all_keys = dict.keys()
    s1 = set(all_keys)
    s2 = set(keys)
    return s1.union(s2) == s1


def write2excel(file_path: str, df: pandas.DataFrame):
    dn = os.path.dirname(file_path)
    if not os.path.exists(dn):
        os.makedirs(dn)
    df.to_csv(file_path, index=False)

In [2]:
import logging
import time

from functools import wraps

def get_logger(dirname):
    import os
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    logger = logging.getLogger('main')
    logger.setLevel(logging.INFO)

    ch = logging.StreamHandler()
    t = time.strftime("%Y-%m-%d %H-%M-%S", time.localtime())
    filename = f'{dirname}/{t}.log'
    fh = logging.FileHandler(filename, mode='a+')
    ch.setLevel(logging.INFO)
    fh.setLevel(logging.INFO)

    msg_fmt = '[{levelname:^4s}] {asctime} moudle {name} function {funcName:16s} +{lineno:<4d} ' \
              ' pid [{thread}] \n {message}'
    tm_fmt = "%Y-%m-%d %H:%M:%S"
    formatter_ch = logging.Formatter(fmt=msg_fmt, style='{', datefmt=tm_fmt)
    formatter_fh = logging.Formatter(fmt=msg_fmt, style='{', datefmt=tm_fmt)

    ch.setFormatter(formatter_ch)
    fh.setFormatter(formatter_fh)

    logger.addHandler(ch)
    logger.addHandler(fh)
    return logger


def LOGGING_INFO(func):
    @wraps
    def wrapper(*args, **kwargs):
        logger = logging.getLogger() 
        logger.info(f'start to run {func.__name__}')
        ret = func(*args, **kwargs)
        logger.info(f'finish running {func.__name__}')
        return ret
    return wrapper


LOGGER = get_logger('logs')


In [3]:
import random


BASE = "https://info.chaindigg.com"

URL_BTC = BASE + "/api/address?coinType=btc&hash={addr}&pageSize=100&pageNumber={pageNumber}&channelId="
URL_TRANSACTION_BTC = BASE + "/api/txn?coinType=btc&hash={hash}&channelId="


URL_ETH = BASE + "/api/api/address?coinType=eth&hash={addr}&pageSize=100&pageNumber={pageNumber}&channelId="
# 交易记录
URL_TRANSACTION_ETH = BASE + "/api/api/txn?coinType=eth&hash={hash}&channelId="
# ETH内部交易
URL_INTERNAL_ETH = BASE +  "/api/api/addressInternal?hash={addr}&pageNumber={pageNumber}&pageSize=100&coinType=eth"
# ETH代币交易汇总
URL_TOKEN_TOTAL = BASE + "/api/api/addressTokenKind?hash={addr}&pageNumber={pageNumber}&pageSize=100&coinType=eth"
# ETH代币交易明细
URL_TOKEN_DETAIL_ETH = BASE + "/api/api/addressToken?hash={addr}&tokenHash={token}&pageNumber={pageNumber}&pageSize=30&coinType=eth"

agents = [
	"Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
	"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
	"Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; InfoPath.3; rv:11.0) like Gecko",
	"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)",
	"Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"
]
headers = {
        'User-Agent': random.choice(agents),
}

proxies = {
 "http": "http://222.74.73.202:42055",
 "https": "http://222.74.73.202:42055"
}

In [5]:
from functools import reduce
import time
import requests
import math
import pandas as pd

from concurrent.futures import ThreadPoolExecutor, as_completed

# from config import *
# from misc import *
# from logger import LOGGER, LOGGING_INFO
from multiprocessing import cpu_count
from requests.exceptions import RequestException, Timeout
from json.decoder import JSONDecodeError

idx = 0

def spider_factory(ctype: str, addr, *args, **kwargs):
    sp = None
    ctype = ctype.lower()
    if ctype == 'btc':
        sp = BtcSpider(addr, *args, **kwargs)
    elif ctype in ('eth', 'usdt'):
        sp = EthSpider(addr, *args, **kwargs)
    else:
        LOGGER.error(f'failed to get type spider. error type: {ctype}')
    return sp


def get_data(url: str, timeout=30) -> dict:
    global idx
    idx += 50
    if (idx+1) % 50 == 0:
        delay(10)
    ret = None
    try:
        resp = requests.get(url, headers=headers, timeout=timeout)
        ret = resp.json()['data']
    except Timeout:
        LOGGER.error(f'time out\nurl:{url}')
        delay(5)
        while 1:
            ret = requests.get(url, headers=headers, timeout=timeout * 2).json().get('data', None)
            if ret:
                break
        if not ret:
            raise
    except RequestException as e:
        LOGGER.error(f'failed to get url {url}')
        LOGGER.error(f'{e}')
    except JSONDecodeError:
        LOGGER.error(f'invalid response\nurl:{url}')
        LOGGER.error(resp.content)
        delay(timeout)
        raise
    except KeyError:
        LOGGER.error(f'{url}\nfailed to get data filed from json data')
        raise
    except Exception:
        raise
    else:
        LOGGER.info(f'succeed to get url {url}')
    return ret

class Spider:
    def __init__(self, addr, thread) -> None:
        self.thread_num = 1
        # lthread if thread else cpu_count() // 2
        self.addr = addr
  
    def run(self):
        raise NotImplementedError
    
    def collect_summary(self):
        raise NotImplementedError

    def collect_transactions(self):
        raise NotImplementedError


class BtcSpider(Spider):
    url_summary = URL_BTC
    url_transaction = URL_TRANSACTION_BTC
    summary_keys = ['hash', 'coinType', 'balance', 'totalTxn', 'totalSend', 'totalReceived']
    transaction_summary_keys = ('txHash', 'confirms', 'receivedStr', 'inputNum', 'outputNum', 'balanceStr', 'blockTime')
    transaction_detail = ('inputVos', 'outputInfos')
    header0 = ['交易哈希', '币类型', '余额', '交易次数', '总发送', '总接受', '最近交易哈希', '最近交易额', '最近交易时间', '首次交易哈希', '首次交易额', '首次交易时间']
    header1 = ['交易哈希', '输入地址', '交易额', '时间']
    header2 = ['交易哈希', '输出地址', '交易额', '时间']
    header3 = ['交易哈希', '确认数', '接受/发送', '输入个数', '输出个数', '余额', '时间']
    ctype = 'BTC'

    def __init__(self, addr: str, thread=None) -> None:
        super().__init__(addr, thread)
        self.transaction_hash = []
        self.transaction_summary = []
        self.summary = []
        self.transactions = []
    
    @timer
    def run(self):
        LOGGER.info(f'starting to run spider, type: {self.ctype} addr: {self.addr}')
        self.collect_summary()
        self.collect_transactions()
        LOGGER.info(f'transaction count: {len(self.transactions)}')
        LOGGER.info(f'transaction summary count: {len(self.transaction_summary)}')
        return self.get_summary(), self.get_trans(), self.get_trans_summary()

    def collect_summary(self):
        addr = self.addr
        summary_url = self.url_summary.format(addr=self.addr, pageNumber=0)
        resp = get_data(summary_url)
        summary = collect_from_dict(resp, self.summary_keys)
        latest_trans = resp['lastTxnVO']
        first_trans = resp['firstTxnVO']
        extra_keys = ['txHash', 'receivedStr', 'blockTime']
        extra_first = collect_from_dict(first_trans, extra_keys)
        extra_latest = collect_from_dict(latest_trans, extra_keys)
        summary.extend(extra_latest)
        summary.extend(extra_first)
        self.summary.extend(summary)
        cur_page = 0
        assert resp.get('page').get('pagePosition')
        page_pos = resp['page']['pagePosition']
        cur_page, _, total_page = page_pos.partition(' / ')
        cur_page, total_page = int(cur_page), int(total_page)
        urls = []
        # 生成交易数据urls
        for page in range(cur_page, total_page+1):
            url = self.url_summary.format(addr=addr, pageNumber=page)
            urls.append(url)
        sum_data = list(map(self.process_trans_summary, urls))
        for item in sum_data:
            self.transaction_summary.extend(item)

    def collect_transactions(self):
        producer = set(self.transaction_hash)
        with ThreadPoolExecutor(max_workers=self.thread_num) as executor:
            while producer:
                fs_hash = {executor.submit(BtcSpider.process_trans_detail, hs): hs for hs in producer}
                for item in as_completed(fs_hash, timeout=60):
                    hs = fs_hash[item]
                    try:
                        dt = item.result()
                    except JSONDecodeError:
                        item.cancel()
                        LOGGER.warning(f'ready to resume collecting transaction hash records {hs}')
                    except Exception:
                        raise
                    else:
                        self.transactions.append(dt)
                        producer.remove(hs)

    def process_trans_summary(self, url) -> list:
        items = get_data(url)['record2sVo']
        ret = []
        for item in items:
            need = collect_from_dict(item, self.transaction_summary_keys)
            ret.append(need)
            self.transaction_hash.append(item['txHash'])
        return ret

    @staticmethod
    def process_trans_detail(hs) -> dict:
        url = BtcSpider.url_transaction.format(hash=hs)
        resp = get_data(url)
        input_lst = resp['inputVos']
        output_lst = resp['outputInfos']
        input_info = map(lambda x: [x['addressHash'], float(x['valueStr'])], input_lst)
        input_info = list(input_info)
        output_info = []
        for item in output_lst:
            tmp = item['outptVo']
            addr = tmp['addressHash']
            val = tmp['valueStr']
            output_info.append([addr, val])
        tm = int(resp['time'])
        tm = time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(tm / 1000))
        input_toal = float(resp['inputTotal'])
        output_toal = float(resp['outputTotal'])
        fee = input_toal - output_toal
        LOGGER.debug(f'processing transaction hash {hs}')
        return {
            'input': input_info,
            'output': output_info,
            'input_toal': input_toal,
            'output_total': output_toal,
            'tm': tm,
            'fee': fee,
            'hash': hs
        }

    def get_trans(self) -> dict:
        tb1, tb2 = [], []
        for item in self.transactions:
            inputs = item['input']
            outputs = item['output']
            hs = item['hash']
            tm = item['tm']
            d1 = [[hs, item[0], item[1], tm] for item in inputs]
            d2 = [[hs, item[0], item[1], tm] for item in outputs]
            tb1.extend(d1)
            tb2.extend(d2)
        tb1 = pd.DataFrame(tb1, columns=self.header1)
        tb2 = pd.DataFrame(tb2, columns=self.header2)
        return {
            'input': tb1,
            'output': tb2
        }
    
    def get_trans_summary(self) -> pd.DataFrame:
        return pd.DataFrame(self.transaction_summary,
                columns=self.header3)
    
    def get_summary(self) -> list:
        return pd.DataFrame([self.summary], columns=self.header0)
    
    def write2excel(self, data, dirname='btc'):
        addr = self.addr
        d1, d2, d3 = data
        p1 = f'{dirname}/{addr}/汇总/汇总.csv'
        p2 = f'{dirname}/{addr}/交易/输入.csv'
        p3 = f'{dirname}/{addr}/交易/输出.csv'
        p4 = f'{dirname}/{addr}/交易/汇总.csv'
        write2excel(p1, d1)
        write2excel(p2,  d2['input'])
        write2excel(p3, d2['output'])
        write2excel(p4, d3)



class EthSpider(Spider):
    url_summary = URL_ETH
    url_transaction = URL_TRANSACTION_ETH
    url_internal = URL_INTERNAL_ETH
    url_token_summary = URL_TOKEN_TOTAL
    url_token_detail = URL_TOKEN_DETAIL_ETH
    summary_keys = ['hash', 'coinType', 'balance', 'txCount', 'totalSend', 'totalReceive', 'firstTime', 'firstTxsValue',  'lastTime', 'lastTxsValue']
    transaction_eth_keys = ['txHash', 'blockHeight', 'fromAddress', 'toAddressHash', 'fee', 'value', 'timeStamp']
    transaction_internal_keys = ['txHash', 'blockHeight', 'from', 'to', 'value', 'timeStamp']
    transaction_token_keys = ['token', 'totalReceive', 'totalSend', 'balance', 'tokenHash']
    token_detail_keys = ['txHash', 'from', 'to', 'value', 'balance', 'timeStamp']

    header0 = ['交易哈希', '币类型', '余额', '交易次数', '总发送', '总接受', '首次交易时间','首次交易额','最近交易时间', '最近交易额']
    header1 = ['交易哈希', '所在区块', '发送地址', '接受地址', '交易费', '接收/发送', '时间']
    header2 = ['交易哈希', '所在区块', '发送', '接受', '接受/发送', '时间']
    header3 = ['代币交易', '总接受', '总发送', '余额', '交易哈希']
    header4 = ['币种', '父区快交易哈希值', '发送', '接受', '接受/发送', '余额', '时间戳']
    ctype = 'ETH'

    def __init__(self, addr: str, thread=None) -> None:
        super().__init__(addr, thread)
        self.transaction_etc_hash = []
        self.summarys = []
        self.transactions = []
    
    @timer
    def run(self):
        LOGGER.info(f'starting to run spider, type: {self.ctype} addr: {self.addr}')
        summary, eth_trans = self.collect_summary()
        inter_trans, token_trans =  self.collect_transactions()
        LOGGER.info(f'eth transaction count: {len(eth_trans)}')
        LOGGER.info(f'internal transaction count: {len(inter_trans)}')
        LOGGER.info(f'token transaction summary count: {len(token_trans["summary"])}')
        LOGGER.info(f'token transaction detail count: {len(token_trans["detail"])}')
        return summary, eth_trans, inter_trans, token_trans['summary'], token_trans['detail']

    def collect_summary(self):
        addr = self.addr
        summary_url = self.url_summary.format(addr=addr, pageNumber=0)
        resp = get_data(summary_url)
        summary = collect_from_dict(resp, self.summary_keys)
        total_page = math.ceil(resp['txCount'] / 100)
        self.summarys.extend(summary)
        urls = [self.url_summary.format(addr=addr, pageNumber=i) for i in range(total_page)]
        eth_data = []
        for url in urls:
            item = get_data(url)['ordinaryTx']
            eth_data.extend(item)
        eth_data = map(lambda x: collect_from_dict(x, EthSpider.transaction_eth_keys), eth_data)
        trans = list(eth_data)
        self.transaction_etc_hash.extend([records[0] for records in trans])
        return summary, trans


    def collect_transactions(self):
        internal_trans = self.collect_internal_trans()
        token_trans = self.collect_token()
        return internal_trans, token_trans

    
    def collect_internal_trans(self) -> list:
        addr = self.addr
        url = EthSpider.url_internal.format(addr=addr, pageNumber=0)
        data = get_data(url).get('transfers', [])
        data = map(lambda x: collect_from_dict(x, EthSpider.transaction_internal_keys), data)
        return list(data)
    

    def collect_token(self) -> list:
        from math import ceil
        summary = self.collect_token_summary()
        addr = self.addr
        symbol_urls = {}
        page_size = 30
        token_transactions = {}
        for item in summary:
            tk_symbol, tk_hash = item[0], item[-1]
            url_first = EthSpider.url_token_detail.format(addr=addr, token=tk_hash, pageNumber=0)
            try:
                page_cnt = ceil(get_data(url_first)['count'] / page_size)
            except Exception as e:
                LOGGER.error(url_first)
                continue
            symbol_urls[tk_symbol] = [EthSpider.url_token_detail.format(addr=addr, token=tk_hash, pageNumber=i) \
                 for i in range(page_cnt+1)]
        for symbo, urls in symbol_urls.items():
            with ThreadPoolExecutor(max_workers=self.thread_num) as executor:
                producer = set(urls)
                lst = []
                while producer:
                    fs_url = {executor.submit(EthSpider.process_token_detail, url): url for url in producer}
                    for item in as_completed(fs_url, timeout=120):
                        url = fs_url[item]
                        try:
                            dt = item.result()
                        except (JSONDecodeError, Timeout):
                            item.cancel()
                            LOGGER.warning(f'ready to resume collecting transaction hash records {url}')
                        except Exception:
                            raise
                        else:
                            lst.extend(dt)
                            producer.remove(url)
            token_transactions[symbo] = lst
        return {
            'summary': summary,
            'detail': token_transactions
        }


    def collect_token_summary(self) -> list:
        url_t3 = EthSpider.url_token_summary.format(addr=self.addr, pageNumber=0)
        data = get_data(url_t3).get('erc20TokenList', [])
        data = map(lambda x: collect_from_dict(x, EthSpider.transaction_token_keys), data)
        return list(data)


    @staticmethod
    def process_token_detail(url) -> list:
        resp = get_data(url)
        tk = url.split('&')[0].partition('=')[-1].lower()
        symbol = resp.get('symbol', None)
        if symbol is None:
            resp = get_data(url, timeout=2*60)
        resp = resp['transfers']
        def map_func(item: dict):
            item['timeStamp'] = time.strftime('%Y/%m/%d %H:%M:%S', time.localtime(item['timeStamp']/1000))
            if tk == item['from']:
                item['value'] = '-' + item['value']
            data = collect_from_dict(item, EthSpider.token_detail_keys)
            return data
        ret = map(map_func, resp)
        ret = map(lambda x: [symbol] + x, ret)
        return list(ret)
    
    def write2excel(self, data, dirname='eth'):
        addr = self.addr
        e0 = f'{dirname}/{addr}/汇总/总计.csv'
        e1 = f'{dirname}/{addr}/交易/交易记录.csv'
        e2 = f'{dirname}/{addr}/交易/内部交易记录.csv'
        e3 = f'{dirname}/{addr}/交易/代币交易汇总.csv'
        e4 = '{dirname}/{addr}/交易/代币交易记录/{type}.csv'

        d0, d1, d2, d3, d4 = data
        d0 = pd.DataFrame([d0], columns=EthSpider.header0)
        d1 = pd.DataFrame(d1, columns=EthSpider.header1)
        d2 = pd.DataFrame(d2, columns=EthSpider.header2)
        d3 = pd.DataFrame(d3, columns=EthSpider.header3)
        write2excel(e0,d0)
        write2excel(e1,d1)
        write2excel(e2,d2)
        write2excel(e3,d3) 

        for ct, dt in d4.items():
            dt = pd.DataFrame(dt, columns=EthSpider.header4)
            write2excel(e4.format(addr=addr, type=ct, dirname=dirname),dt)


In [16]:
import os
os.chdir(r"C:\\Users\\zqy\\Desktop\\Project\tool")

import pandas as pd
import sys

# from misc import *
# from spider import spider_factory
from pprint import pprint

if __name__ == '__main__':
    spiders, lst = [], []
    fn = 'data/clue.xlsx'
    lst = pd.read_excel(fn).values.tolist()
    print('data')
    pprint(lst)

    for idx, item in enumerate(lst):
        addr, ctype = item
        spider = spider_factory(ctype, addr, thread=1)
        if spider:
            spiders.append(spider)
    
    summary_path = '{ctype}/summary/summary.xlsx'

    summarys = []
    for spider in spiders:
        #if (idx+1) % 4 == 0:
        # delay(10)
        data = spider.run()
        spider.write2excel(data, dirname=f'data/{spider.ctype}')

[INFO] 2022-07-05 10:30:03 moudle main function run              +259   pid [15152] 
 starting to run spider, type: ETH addr: 0x8e0b6ab305e8170e489e0941904982ac88bf1e33
[INFO] 2022-07-05 10:30:03 moudle main function run              +259   pid [15152] 
 starting to run spider, type: ETH addr: 0x8e0b6ab305e8170e489e0941904982ac88bf1e33


data
[['0x8e0b6ab305e8170e489e0941904982ac88bf1e33', 'ETH'],
 ['0x28416965e7f89a14a306838c450ad47ae995c7a4', 'ETH'],
 ['0xF141F517D1c31FE71F1192255de6541b61e4e0F0', 'ETH'],
 ['0x94522163b191c9e3387105ce97e732e5b9a7d7bf', 'ETH'],
 ['0xe8D87e10D57ba8CF059652567eAb327679891837', 'ETH'],
 ['0x7c07d50512f329cbaf6422ace1ebff20617b0e98', 'ETH'],
 ['0x943b3f7c10d3051e8499dcfbeb3fcd762981bcae', 'ETH'],
 ['0xda12c12fa7712e4bae15d14fb6739589af79f9e3', 'ETH'],
 ['0x149e13e53957bb3535b7f44c81e05df8e7f3fd15', 'ETH'],
 ['0x4529520f6d8c41a4d0b7bd839011f39e8e78e960', 'ETH'],
 ['0xDFab1CD943c901f79278aEbFed3F5E67b755991f', 'ETH'],
 ['0x50770d6cf636382fe9a8b37aaa54f535cbe0062f', 'ETH']]


[INFO] 2022-07-05 10:30:04 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/address?coinType=eth&hash=0x8e0b6ab305e8170e489e0941904982ac88bf1e33&pageSize=100&pageNumber=0&channelId=
[INFO] 2022-07-05 10:30:04 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/address?coinType=eth&hash=0x8e0b6ab305e8170e489e0941904982ac88bf1e33&pageSize=100&pageNumber=0&channelId=
[INFO] 2022-07-05 10:30:04 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/address?coinType=eth&hash=0x8e0b6ab305e8170e489e0941904982ac88bf1e33&pageSize=100&pageNumber=0&channelId=
[INFO] 2022-07-05 10:30:04 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/address?coinType=eth&hash=0x8e0b6ab305e8170e489e0941904982ac88bf1e33&pageSize=100&pageNumber=0&channelId=
[INFO] 2022-07-0

[INFO] 2022-07-05 10:30:08 moudle main function get_data         +62    pid [13708] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x28416965e7f89a14a306838c450ad47ae995c7a4&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=16&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:30:08 moudle main function get_data         +62    pid [13708] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x28416965e7f89a14a306838c450ad47ae995c7a4&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=0&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:30:08 moudle main function get_data         +62    pid [13708] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x28416965e7f89a14a306838c450ad47ae995c7a4&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=0&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:30:09 moudle main function get_data         +62    pid [13708] 
 succeed to get url https://info.chain

[INFO] 2022-07-05 10:30:13 moudle main function get_data         +62    pid [13708] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x28416965e7f89a14a306838c450ad47ae995c7a4&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=11&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:30:13 moudle main function get_data         +62    pid [13708] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x28416965e7f89a14a306838c450ad47ae995c7a4&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=11&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:30:13 moudle main function get_data         +62    pid [13708] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x28416965e7f89a14a306838c450ad47ae995c7a4&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=10&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:30:13 moudle main function get_data         +62    pid [13708] 
 succeed to get url https://info.cha

...

[INFO] 2022-07-05 10:31:01 moudle main function get_data         +62    pid [16176] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x943b3f7c10d3051e8499dcfbeb3fcd762981bcae&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=19&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:31:01 moudle main function get_data         +62    pid [16176] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x943b3f7c10d3051e8499dcfbeb3fcd762981bcae&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=15&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:31:01 moudle main function get_data         +62    pid [16176] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x943b3f7c10d3051e8499dcfbeb3fcd762981bcae&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=15&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:31:02 moudle main function get_data         +62    pid [16176] 
 succeed to get url https://info.cha

[INFO] 2022-07-05 10:31:06 moudle main function get_data         +62    pid [16176] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x943b3f7c10d3051e8499dcfbeb3fcd762981bcae&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=4&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:31:06 moudle main function get_data         +62    pid [16176] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x943b3f7c10d3051e8499dcfbeb3fcd762981bcae&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=4&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:31:06 moudle main function run              +262   pid [15152] 
 eth transaction count: 118
[INFO] 2022-07-05 10:31:06 moudle main function run              +262   pid [15152] 
 eth transaction count: 118
[INFO] 2022-07-05 10:31:06 moudle main function run              +263   pid [15152] 
 internal transaction count: 0
[INFO] 2022-07-05 10:31:06 moudle main function run              +263   pi

[INFO] 2022-07-05 10:31:09 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/addressTokenKind?hash=0x149e13e53957bb3535b7f44c81e05df8e7f3fd15&pageNumber=0&pageSize=100&coinType=eth
[INFO] 2022-07-05 10:31:09 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/addressTokenKind?hash=0x149e13e53957bb3535b7f44c81e05df8e7f3fd15&pageNumber=0&pageSize=100&coinType=eth
[INFO] 2022-07-05 10:31:09 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x149e13e53957bb3535b7f44c81e05df8e7f3fd15&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=0&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:31:09 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x149e13e53957bb3535b7f44c81e05df8e7f3fd15&tokenHash=0xdac17f9

[INFO] 2022-07-05 10:31:12 moudle main function run              +259   pid [15152] 
 starting to run spider, type: ETH addr: 0xDFab1CD943c901f79278aEbFed3F5E67b755991f
[INFO] 2022-07-05 10:31:13 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/address?coinType=eth&hash=0xDFab1CD943c901f79278aEbFed3F5E67b755991f&pageSize=100&pageNumber=0&channelId=
[INFO] 2022-07-05 10:31:13 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/address?coinType=eth&hash=0xDFab1CD943c901f79278aEbFed3F5E67b755991f&pageSize=100&pageNumber=0&channelId=
[INFO] 2022-07-05 10:31:13 moudle main function get_data         +62    pid [15152] 
 succeed to get url https://info.chaindigg.com/api/api/address?coinType=eth&hash=0xDFab1CD943c901f79278aEbFed3F5E67b755991f&pageSize=100&pageNumber=0&channelId=
[INFO] 2022-07-05 10:31:13 moudle main function get_data         +62    pid [15152] 
 succeed

[INFO] 2022-07-05 10:31:16 moudle main function get_data         +62    pid [17956] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x50770d6cf636382fe9a8b37aaa54f535cbe0062f&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=0&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:31:16 moudle main function get_data         +62    pid [17956] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x50770d6cf636382fe9a8b37aaa54f535cbe0062f&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=0&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:31:17 moudle main function get_data         +62    pid [17956] 
 succeed to get url https://info.chaindigg.com/api/api/addressToken?hash=0x50770d6cf636382fe9a8b37aaa54f535cbe0062f&tokenHash=0xdac17f958d2ee523a2206206994597c13d831ec7&pageNumber=2&pageSize=30&coinType=eth
[INFO] 2022-07-05 10:31:17 moudle main function get_data         +62    pid [17956] 
 succeed to get url https://info.chaind

In [15]:
lst

[['C:\\Users\\zqy\\AppData\\Roaming\\jupyter\\runtime\\kernel-81866824-5df3-4511-9669-ffa90a7deeb5.json',
  '-f']]